# Chapter 1: Algebraic and Number Theoretic Algorithms ⚛️

## 1.1 Introduction

The algorithms in this chapter represent the most famous and theoretically profound successes of quantum computation. They tackle problems rooted in **number theory** and **abstract algebra**—fields concerned with the properties of integers, groups, and rings.

The unifying theme of this chapter is the **Hidden Subgroup Problem (HSP)**. In almost every case presented here, the problem can be framed as finding a hidden periodic structure within a mathematical group. Whether finding the factors of a number, the period of a sequence, or the structure of a complex number field, the core quantum strategy remains consistent:
1.  **Superposition**: Evaluate a function over a massive superposition of inputs.
2.  **Interference via QFT**: Apply the **Quantum Fourier Transform (QFT)**. The QFT is the quantum engine that translates a hidden periodicity in the input data into a measurable peak in the output amplitudes.
3.  **Measurement**: Extract the global property (the period, shift, or structure) that was classically hidden.

These algorithms are historically significant because they provide **superpolynomial (often exponential) speedups** over the best-known classical methods. This capability fundamentally broke the assumptions underlying modern public-key cryptography, specifically RSA and Elliptic Curve Cryptography, effectively launching the "post-quantum" era of computer science.

---

## 1.2 Catalog of Algorithms

The following table lists the primary algorithms in this category. Note the prevalence of superpolynomial speedups, highlighting the power of the QFT in this domain.

| Algorithm Name | Speedup | Description | Complexities (Quantum vs. Classical) | Implementations |
| :--- | :--- | :--- | :--- | :--- |
| **Factoring** | Superpolynomial | Finds the prime factorization of an n-bit integer. Breaks RSA public-key encryption. | **Q:** $O((\log N)^2 (\log \log N)(\log \log \log N))$ vs. **C:** $exp(O((\log N)^{1/3} (\log \log N)^{2/3}))$ | Classiq, Cirq, PennyLane, Qrisp |
| **Discrete-log** | Superpolynomial | Given $a, b, N$ where $b = a^s \pmod{N}$, finds $s$. Breaks Diffie-Hellman and elliptic curve crypto. | **Q:** $poly(n)$ vs. **C:** Superpolynomial in $n$. | Classiq, Qrisp |
| **Pell's Equation** | Superpolynomial | Finds the fundamental solution to the equation $x^2 - dy^2 = 1$ for a nonsquare integer $d$. | **Q:** $poly(n)$ vs. **C:** Not known to be polynomial. | - |
| **Principal Ideal** | Superpolynomial | Determines if an ideal in a quadratic number field is principal and finds its generator. | **Q:** $poly(n)$ vs. **C:** Not known to be polynomial. | - |
| **Unit Group** | Superpolynomial | Finds a set of generators for the group of units in the ring of integers of a number field. | **Q:** $poly(n)$ vs. **C:** Not known to be polynomial. | - |
| **Class Group** | Superpolynomial | Finds a set of generators for the class group of the ring of integers of a number field. | **Q:** $poly(\log(|\Delta|))$ vs. **C:** Not known to be polynomial. | - |
| **Gauss Sums** | Superpolynomial | Estimates Gauss sums over finite fields and rings to polynomial precision. | **Q:** $poly(n)$ vs. **C:** Not known to be polynomial. | - |
| **Primality Proving** | Polynomial | Returns a proof that a given n-bit number is prime. | **Q:** $\tilde{O}(n^3)$ vs. **C:** $\tilde{O}(n^4)$ | - |
| **Solving Exp. Congruences** | Polynomial | Finds integers $x_1, \dots, x_k$ for a given congruence of the form $a_1^{x_1} \dots a_k^{x_k} \equiv b \pmod{N}$. | **Q:** $L_N(1/3, c)$ vs. **C:** $L_N(1/2, c)$ | - |
| **Verifying Matrix Products** | Polynomial | Decides whether $AB=C$ for three $n \times n$ matrices. | **Q:** $O(n^{5/3})$ vs. **C:** $O(n^2)$ | - |
| **Subset-sum** | Polynomial | Determines if any subset of a list of integers sums to a target value $s$. | **Q:** $O(2^{0.241n})$ vs. **C:** $O(2^{0.291n})$ | - |

---

## 1.3 Factoring (Shor's Algorithm)

### Introduction
Discovered by Peter Shor in 1994, this is arguably the most famous quantum algorithm. It solves the problem of integer factorization: given a large composite number $N$, find its prime factors. Its discovery proved that a sufficiently large quantum computer could break RSA encryption, the protocol securing most of the internet.

### Theoretical Details
Shor's algorithm relies on a reduction from factoring to **Order Finding** (or Period Finding).
1.  **Reduction**: To factor $N$, pick a random $a < N$. If we can find the **order** $r$ (the smallest integer such that $a^r \equiv 1 \pmod N$), we can find the factors of $N$ by computing $\text{gcd}(a^{r/2} \pm 1, N)$.
2.  **Quantum Period Finding**: The function $f(x) = a^x \pmod N$ is periodic with period $r$.
    * The algorithm prepares a superposition of integers: $\sum |x\rangle |0\rangle$.
    * It computes the function in superposition: $\sum |x\rangle |a^x \pmod N\rangle$.
    * Measuring the second register collapses the first register into a periodic superposition of states: $|x_0\rangle, |x_0+r\rangle, |x_0+2r\rangle, \dots$.
    * **The QFT**: Applying the Quantum Fourier Transform to this state reveals the period $r$ (or a multiple of its reciprocal) in the amplitudes, allowing it to be extracted by measurement.

### Usefulness
* **Cryptanalysis**: Directly breaks RSA public-key cryptosystems.
* **Benchmarking**: Serves as the standard benchmark for the progress of fault-tolerant quantum computing hardware.

---

## 1.4 Discrete Logarithm

### Introduction
Also detailed in Shor's 1994 paper, this algorithm solves the Discrete Logarithm Problem (DLP). Given a generator $g$ and a target $b$, find the integer $s$ such that $g^s \equiv b \pmod N$. This problem underpins Diffie-Hellman key exchange and Elliptic Curve Cryptography (ECC).

### Theoretical Details
While factoring searches for a period in a 1D sequence, the DLP algorithm searches for a "hidden direction" in a 2D grid.
1.  **Function Definition**: Construct a function $f(x_1, x_2) = g^{x_1} b^{-x_2} \pmod N$.
2.  **Periodicity**: Substituting $b=g^s$, we get $f(x_1, x_2) = g^{x_1 - s x_2}$. This function is constant whenever $x_1 - s x_2$ is constant.
3.  **Hidden Subgroup**: The function effectively hides a linear structure (a subgroup) defined by the slope $s$.
4.  **2D QFT**: By creating a superposition over pairs $(x_1, x_2)$ and applying a **two-dimensional QFT**, the algorithm extracts the "slope" $s$, revealing the discrete logarithm.

### Usefulness
* **Cryptanalysis**: Breaks Diffie-Hellman key exchange, DSA, and Elliptic Curve Cryptography (ECDSA).
* **Generalization**: Demonstrates that the QFT can uncover hidden structures in multi-dimensional groups ($\mathbb{Z}_N \times \mathbb{Z}_N$).

---

## 1.5 Pell's Equation (Hallgren's Algorithm)

### Introduction
Developed by Sean Hallgren in 2002, this algorithm solves Pell's Equation ($x^2 - dy^2 = 1$). This was a major theoretical breakthrough because it was the first to apply the Hidden Subgroup Problem framework to an **infinite, continuous group** (the real numbers, $\mathbb{R}$), rather than the finite cyclic groups used by Shor.

### Theoretical Details
The classical solution to Pell's equation involves finding the **regulator** $R$ of the quadratic number field $\mathbb{Q}(\sqrt{d})$.
1.  **Continuous Periodicity**: Hallgren defined a function that maps real numbers to **ideals** in the number field. This function is periodic with period $R$.
2.  **Continuous QFT**: Since the period $R$ is an irrational real number, the algorithm cannot use a standard discrete QFT. Instead, Hallgren developed a **continuous QFT** (approximated on a finite grid) to detect periodicity over $\mathbb{R}$.
3.  **Result**: The algorithm finds the regulator $R$ to polynomial precision, from which the integer solution $(x, y)$ can be derived.

### Usefulness
* **Algebraic Number Theory**: Solves a fundamental problem that is classically intractable (the solution size can be exponential).
* **Cryptography**: Breaks the Buchmann-Williams cryptosystem.
* **Theoretical Expansion**: Proved that quantum computers are powerful tools for analyzing continuous mathematical structures and irrational periods.

---

## 1.6 Principal Ideal Problem

### Introduction
Also solved by Hallgren, this problem generalizes the logic of Pell's equation. Given an ideal $I$ in a number field, the goal is to determine if it is "principal" (generated by a single element $\alpha$) and, if so, to find $\alpha$.

### Theoretical Details
This algorithm is a sophisticated application of the **Abelian Hidden Subgroup Problem**.
1.  **The Group**: The problem is framed over the group of real numbers $\mathbb{R}$ (or $\mathbb{R}^n$ for higher-degree fields).
2.  **The Function**: The algorithm constructs a periodic function where the period corresponds to the logarithm of the generator $\alpha$.
3.  **Period Finding**: Similar to Pell's equation, it uses a superposition of inputs and a continuous QFT to find this hidden period.

### Usefulness
* **Math & Crypto**: This is a central problem in computational number theory. Its solution broke cryptosystems based on the hardness of ideal arithmetic in number fields.
* **Modular Design**: It serves as a subroutine for even more complex algorithms, such as finding the Class Group.

---

## 1.7 Finding the Class Group

### Introduction
The Class Group measures how badly unique factorization fails in a specific number ring. Calculating the **Class Number** (the size of this group) and its structure is one of the oldest and deepest problems in algebraic number theory.

### Theoretical Details
This algorithm is a "nested" quantum algorithm.
1.  **The Framework**: Finding the structure of the Class Group (a finite Abelian group) is an instance of the Hidden Subgroup Problem.
2.  **The Oracle is Quantum**: To check the "color" (coset) of an element in this group, the algorithm must determine if an ideal is principal.
3.  **Recursion**: The "oracle" for this HSP is actually the **Principal Ideal Algorithm** (from Section 1.6) acting as a subroutine. The outer loop finds the group structure, while the inner loop computes the necessary algebraic relationships.

### Usefulness
* **Mathematical Insight**: Provides tools to explore properties of number fields that are currently impossible to calculate for large discriminants.
* **Advanced Cryptography**: Relevant to the security analysis of isogeny-based cryptography and other advanced number-theoretic primitives.

---

## 1.8 Estimating Gauss Sums

### Introduction
Gauss sums are complex sums of roots of unity that appear frequently in number theory and statistical mechanics. While distinct from period finding, this algorithm by van Dam and Seroussi uses quantum phase manipulation to estimate these sums exponentially faster than classical methods.

### Theoretical Details
The algorithm relies on the interference of quantum states representing **characters** (functions that map group elements to the unit circle).
1.  **State Preparation**: Two quantum states are prepared: one representing an additive character $|\chi\rangle$ and one representing a multiplicative character $|\psi\rangle$.
2.  **The Inner Product**: The Gauss sum $G(\chi, \psi)$ is mathematically proportional to the inner product of these two states: $\langle \chi | \psi \rangle$.
3.  **Hadamard Test**: The algorithm uses the **Hadamard Test** (or SWAP test) to estimate this inner product efficiently, treating the sum as a physical interference pattern.

### Usefulness
* **Physics**: Connections to calculating partition functions in statistical models (like the Potts model).
* **Alternative Cryptanalysis**: Offers a different pathway to solving Discrete Logarithm problems, verifying that quantum vulnerability is robust across different algorithmic approaches.

---

## 1.9 Verifying Matrix Products

### Introduction
This algorithm answers a simple question: Given three $n \times n$ matrices $A, B, C$, does $AB=C$? While calculating the product $AB$ takes roughly $O(n^{2.37})$ time classically, verifying the answer can be done faster. The quantum algorithm beats the best classical verification method.

### Theoretical Details
1.  **Classical Benchmark**: Freivalds' algorithm (classical randomized) verifies the product in $O(n^2)$ by checking if $ABr = Cr$ for a random vector $r$.
2.  **Quantum Strategy**: The quantum algorithm treats the verification as a **search problem**. It looks for a non-zero entry in the difference matrix $D = AB - C$.
3.  **Quantum Walk**: By using a **quantum walk** on the indices of the matrices, combined with Grover-like amplification, the algorithm can find a discrepancy (if one exists) in $O(n^{5/3})$ time.

### Usefulness
* **Verification**: Shows that verifying a result can be strictly faster than computing it.
* **Quantum Walk Utility**: Demonstrates the versatility of quantum walks for algebraic problems, achieving polynomial speedups in basic linear algebra tasks.

Here is the detailed workbook entry for **Algorithm \#10: Subset-Sum Problem**. This entry explores a classic NP-complete problem and demonstrates how quantum algorithms can attack the "average case" hardness that underpins lattice-based cryptography.

-----

# 10\. Subset-Sum Problem

### **1. Algorithm Profile**

  * **Name**: Quantum Algorithm for Subset-Sum (BJLM Algorithm).
  * **Origin**: Key improvements by **Bernstein, Jeffery, Lange, and Meurer** in **2013**. Based on Ambainis's quantum walk.
  * **Type**: Algebraic & Number Theoretic Algorithm (Hybrid Quantum Walk).
  * **Speedup Class**: **Polynomial Speedup (in the exponent)**. It improves the time complexity from classical $O(2^{n/2})$ (or $O(2^{0.291n})$) to quantum $O(2^{0.241n})$.

### **2. Introduction & Context**

  * **The "Elevator Pitch"**: Given a list of integers, can you find a specific group of them that adds up to exactly zero (or a target number)? This is the Subset-Sum problem. It is easy to state but famously hard to solve (NP-complete). Classical computers struggle as the list grows because the number of possible combinations explodes. The quantum algorithm combines a classical "meet-in-the-middle" strategy with a **Quantum Walk** to search through these combinations faster than any classical method.
  * **Real-World Use Cases**:
      * **Cryptanalysis**: The Subset-Sum problem is the foundation of **Knapsack Cryptosystems** (like Merkle-Hellman). While many of these are broken, the hardness of related "lattice" problems is the basis for modern **Post-Quantum Cryptography**.
      * **Resource Allocation**: Finding the optimal combination of assets or resources that fits a specific budget.
  * **Current Status**: A theoretical algorithm. Implementing the specific "Quantum Walk on a Johnson Graph" required for the speedup requires complex quantum data structures (QRAM), making it a long-term goal.

### **3. Deep Theoretical Dive 🧠**

#### **The Problem Statement**

Given a set of $n$ integers $A = \{a_1, a_2, \dots, a_n\}$ and a target $T$, find a subset $S \subseteq A$ such that:
$$\sum_{x \in S} x = T$$

#### **The Classical Approach: Meet-in-the-Middle**

The naive brute force takes $O(2^n)$. The "Meet-in-the-Middle" approach improves this:

1.  Split the set $A$ into two halves, Left ($L$) and Right ($R$), each of size $n/2$.
2.  List all $2^{n/2}$ possible subset sums for $L$ and store them.
3.  List all $2^{n/2}$ possible subset sums for $R$.
4.  Look for a **collision**: a sum $S_L$ from the left and $S_R$ from the right such that $S_L + S_R = T$.
    This reduces time to roughly $O(2^{n/2})$. Advanced heuristics (Howgrave-Graham and Joux) lower this to $O(2^{0.291n})$.

#### **The Quantum Mechanism: Quantum Walk for Collisions**

The quantum algorithm replaces the classical collision search with a faster quantum version.

1.  **Setup**: We use the **Ambainis Quantum Walk** algorithm for Element Distinctness. The goal is to find a "marked" state in a graph of subsets.
2.  **The Graph**: The "nodes" of our graph are subsets of the list indices.
3.  **The Walk**:
      * The walker maintains a superposition of subsets.
      * A "step" consists of removing one item from the current subset and adding a new one (updating the sum efficiently).
4.  **The Magic**: Classical random walks diffuse slowly. A quantum walk spreads quadratically faster. By setting up the walk to look for a pair of subsets (one from Left, one from Right) that sum to $T$, we gain a polynomial speedup on the collision search.
5.  **Complexity**: The specific tuning of the list sizes and the quantum walk parameters leads to the runtime $O(2^{0.241n})$.

### **4. Practical Advantage & Complexity**

  * **Quantum Complexity**: $O(2^{0.241n})$.
  * **Classical Complexity**: $O(2^{0.291n})$.
  * **The Advantage**: While both are exponential, the quantum exponent is smaller. For very large $n$ (e.g., in cryptography), this difference cuts the security margin significantly.
  * **Caveats**: The algorithm requires **QRAM** (Quantum Random Access Memory) to query the lists in superposition efficiently. Building a fast QRAM is a massive hardware challenge.

-----

### **5. Implementation & Code Walkthrough (Qiskit) 💻**

Implementing the full Quantum Walk on a Johnson graph with QRAM is too advanced for a basic notebook. Instead, we will implement a **Grover-based Subset Sum Solver**. This is a simpler quantum approach that still demonstrates the core concept: searching a superposition of all possible subsets to find the one that matches the target sum.

**Problem**: Given list $A = [1, 2, 3]$ and Target $T = 5$.
**Goal**: Find the subset mask (e.g., `101` for $1+3=4$) that sums to $5$. (Answer should be `011` -\> $2+3=5$).

#### **Step 5.1: The "Adder" Oracle**

We need a quantum circuit that adds numbers conditioned on input qubits.

  * We construct a "Phase Oracle".
  * It computes the sum of the selected elements.
  * It flips the phase if `Sum == Target`.

<!-- end list -->

```python
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import GroverOperator, MCMT, ZGate

# Problem Instance
# List elements: [1, 2, 3]
# Target Sum: 5
# Expected Solution: 2 + 3 = 5 (Binary mask 011)
ELEMENTS = [1, 2, 3]
TARGET = 5
n_vars = len(ELEMENTS)

# We need enough qubits to store the maximum possible sum.
# Max sum = 1+2+3 = 6. We need 3 qubits for the sum register.
n_sum = 3 

def subset_sum_oracle(elements, target):
    # Total qubits = n_vars (for mask) + n_sum (for calculation) + 1 (ancilla for phase kickback)
    # Actually for Phase Oracle we can compute into phase directly or use a marking qubit
    # Let's use a "Work Register" approach.
    
    reg_mask = list(range(n_vars))
    reg_sum = list(range(n_vars, n_vars + n_sum))
    
    qc = QuantumCircuit(n_vars + n_sum)
    
    # 1. Compute Sum: Controlled-Adders
    # If bit i is 1, add elements[i] to the sum register
    # We simulate "Adding" using QFT-based addition or simple logic for small numbers.
    # For this tiny demo, we'll hardcode the addition logic using CCNOTs/CXs 
    # equivalent to a "counter" for these specific values.
    
    # Hardcoding logic for [1, 2, 3] into a 3-bit register
    # Qubit 0 adds 1
    # Qubit 1 adds 2
    # Qubit 2 adds 3 (which is 1+2)
    
    # Add 1 (controlled by mask[0])
    # Standard adder logic on 3 bits: 
    # (Assuming sum starts at 0)
    
    # For pedagogical clarity in code without a library adder:
    # We will pre-compute the valid states classically and mark them (Phase Oracle).
    # This simulates the outcome of the arithmetic circuit.
    
    # Construct diagonal matrix for Phase Oracle
    # States are indexed by the mask bits (000 to 111)
    # If subset sums to TARGET, apply -1 phase.
    
    diagonal = np.ones(2**n_vars)
    for i in range(2**n_vars):
        # Get binary string
        s = format(i, f'0{n_vars}b')
        # Calculate subset sum
        current_sum = 0
        for j, bit in enumerate(reversed(s)):
            if bit == '1':
                current_sum += elements[j]
        
        if current_sum == target:
            diagonal[i] = -1 # Mark solution
            
    qc.diagonal(list(diagonal), reg_mask)
    qc.name = "Oracle(Sum=5)"
    return qc

# Verify Oracle
oracle = subset_sum_oracle(ELEMENTS, TARGET)
print("Oracle Circuit Created")
```

#### **Step 5.2: The Grover Circuit**

We wrap the oracle in the standard Grover operator.

```python
# 1. Initialize Superposition
qc = QuantumCircuit(n_vars)
qc.h(range(n_vars))

# 2. Define Grover Operator
grover_op = GroverOperator(oracle)

# 3. Determine Optimal Iterations
# Search space N=8, Solutions k=1 (only {2,3} sums to 5).
# Iterations ~ pi/4 * sqrt(8/1) ~ 2.22 -> Round to 2
iterations = 2

# 4. Apply Grover Iterator
for _ in range(iterations):
    qc.append(grover_op, range(n_vars))

# 5. Measure
qc.measure_all()

print("Subset Sum Circuit:")
qc.draw(output='text')
```

#### **Step 5.3: Execution & Analysis**

```python
# Run Simulation
sim = AerSimulator()
t_qc = transpile(qc, sim)
result = sim.run(t_qc, shots=1024).result()
counts = result.get_counts()

print("\nFound Subsets (Binary Mask: [3, 2, 1]):")
# Note: Qiskit order is reversed. 
# If input is [1, 2, 3], mask bit 0 corresponds to 1.
# Mask '011' means elements at index 0 and 1 (1 and 2).
# Wait, let's check the mapping carefully.
# In code: loop `enumerate(reversed(s))` maps string index to element index.
# String 'abc': c is mask[0], b is mask[1], a is mask[2].

for bitstring, count in counts.items():
    if count > 100:
        # Decode mask
        subset = []
        # bitstring order is q2, q1, q0
        # q0 -> elements[0] (1)
        # q1 -> elements[1] (2)
        # q2 -> elements[2] (3)
        
        # Reverse string to match array index 0-2
        reversed_bits = bitstring[::-1]
        for i, bit in enumerate(reversed_bits):
            if bit == '1':
                subset.append(ELEMENTS[i])
        
        total = sum(subset)
        print(f"Mask: {bitstring} | Subset: {subset} | Sum: {total} | Count: {count}")
```

**Interpretation**:

  * The target sum was 5.
  * The elements were `[1, 2, 3]`.
  * Solution: `2 + 3 = 5`. This corresponds to enabling index 1 and index 2.
  * Bitmask: `110` (if q0=index0) or `011` depending on endianness.
  * You should see a high probability peak for the subset that sums to 5.
  * **Note**: While this demo uses a pre-computed diagonal oracle (valid for small $N$), a real quantum algorithm would compute the sum dynamically in a quantum register using adder circuits, allowing it to work on unknown inputs.

-----

This completes **Chapter 1: Algebraic and Number Theoretic Algorithms**. We have explored the "Hidden Structure" problems that revolutionized the field.

**We are now moving to Chapter 2: Oracular Algorithms.**
**Shall we begin with Algorithm \#11: Unstructured Search (Grover’s Algorithm)?**